# look into:
 - look into tdnn
 - convs for discrim
 - lstms for discrim
 - convs for gen
 - lstms for gen

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader

In [2]:
class sinDataLoader(Dataset):
    def __init__(self,start,end):
        self.start = start
        self.end = end
        self.pspace = np.arange(start,end+1)
    def return_sin_wave(self, sinmod=1, offset = np.pi*2,step = 1/1000):
        x = np.linspace(0,int(offset),int(1/step))
        y = np.sin(x*(sinmod*step))
        return y
    def __len__(self):
        return len(self.pspace)
    def __getitem__(self,index):
        return self.return_sin_wave(sinmod = self.pspace[index])

In [3]:
dl = DataLoader(sinDataLoader(100,4000),batch_size = 16,shuffle = True)

In [4]:
# plt.plot(next(iter(dl))[8])
# next(iter(dl))[0].shape

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
import torch


In [6]:
in_shape = 1000

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(in_shape, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512*2, 512*3),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512*3, 512*3),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512*3, 1),
        )

    def forward(self, inp):
        validity = self.model(inp)
        return validity


In [8]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat,0.8))
            layers.append(nn.LeakyReLU(.2, inplace=True))
            return layers

        self.modelL = nn.Sequential(
            *block(1, 128, normalize=False),
            *block(128,128*4),
            *block(128*4, 256*4),
            *block(256*4, 512*4),
            *block(512*4, 1024*4),
            *block(1024*4,1024*8),
            nn.Linear(1024*8,1024*16),
            nn.Linear(1024*16, in_shape),
            nn.Tanh()
        )
        self.modelS = nn.Sequential(
            *block(1, 64, normalize=False),
            *block(64,128),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, in_shape),
            nn.Tanh()
        )
        self.ct1 = nn.ConvTranspose1d(in_channels=1,out_channels=1,kernel_size=3,stride=1,padding=1)
        self.ct2 = nn.ConvTranspose1d(1,8,4,2,1)
        self.cv1 = nn.Conv1d(8,1,4,2,1)
    def forward(self,inp):
            return self.modelL(inp)


In [9]:
lambda_gp = 10
cuda = torch.cuda.is_available()

In [10]:
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()


In [11]:

optimizer_G = torch.optim.Adam(
    generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(
    discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [12]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
import sys

In [13]:
def innoise(shape):
    return Variable(torch.abs(Tensor(np.random.normal(
            1.1, 1, (shape[0], 1)))))

In [17]:
from icecream import ic

In [32]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1,).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. 
    ic(d_interpolates.shape,interpolates.shape,fake.shape)
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty



In [33]:
batches_done = 0
n_epochs = 10
for epoch in range(n_epochs):
    for i, imgs in enumerate(dl):
        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = innoise(imgs.shape)

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # -----------------
        #  Train Generator
        # -----------------

        # Generate a batch of images
        fake_imgs = generator(z)
        # Loss measures generator's ability to fool the discriminator
        # Train on fake images
        fake_validity = discriminator(fake_imgs)
        g_loss = -torch.mean(fake_validity)

        g_loss.backward()
        optimizer_G.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dl), d_loss.item(), g_loss.item())
        )
    batches_done+=1

ic| d_interpolates.shape: torch.Size([16, 1, 16, 1])
    interpolates.shape: torch.Size([16, 1, 16, 1000])
    fake.shape: torch.Size([16, 1])


RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([16, 1]) and output[0] has a shape of torch.Size([16, 1, 16, 1]).